In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Flatten, Conv2D, MaxPooling2D, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
from PIL import Image
import tensorflow as tf
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau

In [2]:
# def train_val_spliter(dirname, foldername, classes, rate=0.8):
#   count = 0
#   datapath = os.getcwd()
#   i = 0
#   for (path, dic, files) in os.walk(dirname):
#     if path.split('\\')[-1] == classes:
#       number = len(files)
#       for filename in files:
#         ext = os.path.splitext(filename)[-1]
#         if ext == '.jpg':
#           if count < number * rate:
#             image = cv2.imread(dirname +'/'+classes+'/'+filename)
#             cv2.imwrite(datapath + '/' + foldername + '/' + 'train' + '/' + classes + '/' + filename, image)
#           else:
#             image = cv2.imread(dirname + '/' +classes+'/' +filename)
#             cv2.imwrite(datapath + '/' + foldername + '/' + 'valid' + '/' + classes + '/' + filename, image)
#         count += 1
# def makedirs(dirname, classes):
#   path = os.getcwd()
#   if not os.path.isdir(path +'/' + dirname):
#     os.makedirs(path + '/' + dirname)
#   if not os.path.isdir(path +'/' + dirname + '/' + 'train'):
#     os.makedirs(path + '/' + dirname + '/' + 'train')
#   if not os.path.isdir(path +'/' + dirname + '/' + 'valid'):
#     os.makedirs(path + '/' + dirname + '/' + 'valid')
#   if not os.path.isdir(path +'/' + dirname + '/' + 'train' + '/' + classes):
#     os.makedirs(path +'/' + dirname + '/' + 'train' + '/' + classes)
#   if not os.path.isdir(path +'/' + dirname + '/' + 'valid' + '/' + classes):
#     os.makedirs(path + '/' + dirname + '/' + 'valid' + '/' + classes)

In [3]:
# makedirs('dev_image', 'dog')
# makedirs('dev_image', 'elephant')
# makedirs('dev_image', 'giraffe')
# makedirs('dev_image', 'guitar')
# makedirs('dev_image', 'horse')
# makedirs('dev_image', 'house')
# makedirs('dev_image', 'person')

# path = 'C:\\Users\\subak\\PycharmProjects\\project20210517\\dev_machinelearning\\train'
# train_val_spliter(path, 'dev_image', 'dog')
# train_val_spliter(path, 'dev_image', 'elephant')
# train_val_spliter(path, 'dev_image', 'giraffe')
# train_val_spliter(path, 'dev_image', 'guitar')
# train_val_spliter(path, 'dev_image', 'horse')
# train_val_spliter(path, 'dev_image', 'house')
# train_val_spliter(path, 'dev_image', 'person')

In [4]:
def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend(['Train', 'val'], loc=0)

def plot_acc(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.legend(['Train', 'val'], loc=0)

In [5]:
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, Input, Conv3D, Add, LeakyReLU,AveragePooling2D
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import SGD
# import matplotlib.pyplot as plt
# from tensorflow.keras.regularizers import l2


# def residual_block(x, filters_in, filters_out, k_size):
#     shortcut = x
#     x = Conv2D(filters_in, kernel_size=(1, 1), strides=(1, 1), padding="same")(x)
#     x = BatchNormalization()(x)
#     x = LeakyReLU()(x)

#     x = Conv2D(filters_in, kernel_size=(k_size, k_size), strides=(1, 1), padding="same")(x)
#     x = BatchNormalization()(x)
#     x = LeakyReLU()(x)

#     x = Conv2D(filters_out, kernel_size=(1, 1), strides=(1, 1), padding="same")(x)
#     x = BatchNormalization()(x)

#     shortcut_channel = shortcut.shape.as_list()[-1]

#     if shortcut_channel != filters_out:
#         shortcut = Conv2D(filters_out, kernel_size=(1, 1), strides=(1, 1), padding="same")(shortcut)

#     x = Add()([x, shortcut])
#     return LeakyReLU()(x)

# def define_model(image_height, image_width, image_channel, classes):
#     input_tensor = Input(shape=(image_height, image_width, image_channel))
#     x = Conv2D(64, kernel_size=(7, 7), strides=2, padding='same',
#                kernel_initializer='he_normal')(input_tensor)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
#     x = residual_block(x, 64, 256, 3)
#     x = MaxPooling2D(pool_size=(2, 2))(x)
#     x = residual_block(x, 128, 512, 3)
#     x = AveragePooling2D(pool_size=(2,2))(x)
#     x = Flatten()(x)
#     x = Dropout(0.5)(x)
#     output_tensor = Dense(classes, activation='softmax')(x)
#     model = Model(inputs=input_tensor, outputs=output_tensor)
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model

In [6]:
# def define_model(image_height, image_width, image_channel, classes):
#   input = Input(shape=(image_height, image_width, image_channel))
#   model = ResNet50(input_tensor=input, include_top=False, weights='imagenet', pooling='max')
 
#   x = model.output
#   x = Dense(1024)(x)
#   x = BatchNormalization()(x)
#   x = Activation('relu')(x)
#   x = Dense(512)(x)
#   x = BatchNormalization()(x)
#   x = Activation('relu')(x)
#   x = Dropout(0.5)(x)
#   x = Dense(classes, activation='softmax')(x)
#   model = Model(model.input, x)
#   model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#   return model

In [7]:
def define_model(image_height, image_width, image_channel, classes):
  input = Input(shape=(image_height, image_width, image_channel))
  model = ResNet50(input_tensor=input, include_top=False, weights='imagenet', pooling='max')
  additional_model = Sequential()
  additional_model.add(model)
  additional_model.add(Flatten())
  additional_model.add(Dense(2048, activation='relu'))
  additional_model.add(Dense(1024, activation='relu'))
  additional_model.add(Dropout(0.5))
  additional_model.add(Dense(7, activation='softmax'))
  additional_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return additional_model

In [8]:
model = define_model(227, 227, 3, 7)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              4196352   
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 7175      
Total params: 29,889,415
Trainable params: 29,836,295
Non-trainable params: 53,120
_______________________________________

In [9]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1.0 / 255.0,
                                   shear_range=20,featurewise_std_normalization=True, rotation_range = 90, vertical_flip=True, brightness_range=[0.2, 1.0], zoom_range=[0.5, 1.5])
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_it = train_datagen.flow_from_directory('/content/drive/MyDrive/dev_machinelearning/train',
                                       class_mode='categorical', batch_size=32, target_size=(227, 227))

Found 1652 images belonging to 7 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [10]:
# from tensorflow.keras.models import load_model
# model = load_model('3-1.34.h5')

In [11]:
# model_checkpoint = ModelCheckpoint(filepath='./{epoch}-{val_loss:.2f}.h5', monitor='val_loss', save_best_only=True, verbose=1)
# reduceLR = ReduceLROnPlateau(
#     monitor='val_loss',  # 검증 손실을 기준으로 callback이 호출됩니다
#     factor=0.5,          # callback 호출시 학습률을 1/2로 줄입니다
#     patience=10         # epoch 10 동안 개선되지 않으면 callback이 호출됩니다
# )

In [ ]:
history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
                              validation_data=val_it, validation_steps=len(val_it), epochs=100, verbose=1, callbacks=[model_checkpoint,reduceLR])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/100
52/52 [==============================] - 231s 4s/step - loss: 7.4097 - accuracy: 0.2034
Epoch 2/100
52/52 [==============================] - 27s 514ms/step - loss: 1.9463 - accuracy: 0.2258
Epoch 3/100
52/52 [==============================] - 27s 515ms/step - loss: 1.8779 - accuracy: 0.2476
Epoch 4/100
52/52 [==============================] - 27s 515ms/step - loss: 1.8587 - accuracy: 0.2597
Epoch 5/100
52/52 [==============================] - 27s 521ms/step - loss: 1.7758 - accuracy: 0.3172
Epoch 6/100
36/52 [===================>..........] - ETA: 8s - loss: 1.7746 - accuracy: 0.3211

In [ ]:
model.save('dev_machine_resnet_model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def search(dirname):
    img_list = []
    for (path, dir, files) in os.walk(dirname):
        for filename in files:
            ext = os.path.splitext(filename)[-1]
            if ext == '.jpg':
                img_list.append(path+'/'+filename)
    return img_list

# load and prepare the image
def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(227, 227))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = np.expand_dims(img, axis=0)
    # img scaling
    img /= 255

    return img

In [ ]:
img_list_1 = search('/content/drive/MyDrive/dev_machinelearning/test/0')

model = load_model('./dev_machine_model.h5')



In [ ]:
import csv

with open('/content/drive/MyDrive/test_answer_sample_.csv', 'w') as csvfile:
    cwriter = csv.writer(csvfile)

    for i in range(len(img_list_1)+1):
      if i == 0:
        cwriter.writerow([' ','answer value'])
      else:
        img = load_image(img_list_1[i-1])
        result = model.predict(img)
        preds_value = np.argmax(result, axis=-1)[0]
        cwriter.writerow([i-1, preds_value])

